In [2]:
import os
import pandas as pd
import re
import numpy as np
import glob

from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from tensorflow import keras

from keras.preprocessing.text import Tokenizer

from keras_preprocessing.sequence import pad_sequences

import matplotlib.pyplot as plt

import json

In [49]:
# This was used to train and read on the the open source sample code
train_sections = []

with open('../sample/sample.json', 'r') as f:
  train_data = json.load(f)


for file in train_data:
    # print(file['name'], len(file['sections']))
    for section in file['sections']:
        train_sections.append(section['raw'])

with open('../../data/cobols.json', 'r') as f:
  data = json.load(f)

sections = []
importances = []

for file in data:
    # print(file['name'], len(file['sections']))
    for section in file['sections']:
        sections.append(section['raw'])

print("train: ", len(train_sections), "predict: ", len(sections))

train:  687 predict:  1159


In [50]:
train_data = pd.DataFrame({"section": train_sections}, columns=["section"])
data = pd.DataFrame({"section": sections}, columns=["section"])

regex_target = "[^a-zA-z0-9\s\-]" # include -
# regex_target = "[^a-zA-z0-9\s]" # exclude -

train_data['section'] = train_data['section'].apply((lambda x: re.sub(regex_target,'',x)))
data['section'] = data['section'].apply((lambda x: re.sub(regex_target,'',x)))

max_fatures = 2000
tokenizer =  Tokenizer(num_words=max_fatures, split=' ', filters='-!"#$%&()*+,./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(train_data['section'].values)
X = tokenizer.texts_to_sequences(data['section'].values)
X = pad_sequences(X)


# data
for i in range(11):
    word = list(tokenizer.word_index.keys())[i]
    count = tokenizer.word_counts[word]
    print(word, count)




to 1938
move 1595
exit 1361
map 1114
ws 962
send 928
end 872
exec 868
3030 714
perform 708
thru 696


In [51]:
model = keras.models.load_model('../model/model_open')


In [62]:
predits = model.predict(X)

important_predicts = []

for i in range(len(predits)):
    item = predits[i]
    value = item[1]
    if value > 0.5:
        important_predicts.append((True, value))
    else:
        important_predicts.append((False, 1 - value))




37/37 [==============================] - 1s 20ms/step


In [66]:
i=0

with open('../../data/cobols.json', 'r') as f:
  data = json.load(f)
for file in data:
    # print(file['name'], len(file['sections']))
    for section in file['sections']:
        section['importance'] = important_predicts[i][0]
        section['confidence'] = float(important_predicts[i][1])
        i += 1

with open('../../data/cobols_predicts.json', 'w') as f:
    json.dump(data, f, indent=2)